# PharMeBINet Animation
Sometimes, expecially when preparing a presentation for a conference or your work colleagues, a good animation can say more than a thousand words.

For this reason, we have prepared a straighforward way to create animations for a number of tasks using GRAPE that, through subsampling, can be executed on graphs of arbitrary size.

In this brief tutorial, we will show how to get a PharMeBINet embedding using First-order LINE, and then we will use TSNE decoposition to reduce its dimensionality and plot it into a short video.

The resulting WEBM can be converted using one of many services and can be directly incorporated in Google Slides.

## Installing 🍇 GRAPE

To install the [GRAPE](https://github.com/AnacletoLAB/grape) library, as usual just run:

```bash
pip install grape -U
```

### Checking the library version
To check the library version and its main dependencies, you can use the `print_version` method:

In [1]:
from grape import print_version

print_version()

,Version
Information,
GRAPE Version,0.1.21
Python version,3.9.7
Platform,Linux-5.15.25-1-MANJARO-x86_64-with-glibc2.35
Threads number,24
KarateClub version,1.3.0
PyTorch version,1.11.0+cu102
TensorFlow version,2.8.0
PyKEEN version,1.8.1


## Retrieving PharMeBINet
First, we retrieve PharMeBINet:

In [8]:
from grape.datasets.zenodo import PharMeBINet

graph = PharMeBINet(directed=True)

Then, let's take a look at its graph report:

In [9]:
graph

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>PharMeBINet</h2><p class="paragraph">The directed multigraph PharMeBINet has 2.65M nodes and 15.88M heterogeneous edges. The RAM requirements for the nodes and edges data structures are 214.43MB and 42.64MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 0, the maximum node degree is 34.28K, the mode degree is 0, the mean degree is 5.98 and the node degree median is 0.</p><p class="paragraph">The nodes with the highest degree centrality are 31816 (degree 34.28K), 30205 (degree 24.82K), 798 (degree 23.85K), 868 (degree 23.58K) and 902 (degree 23.10K).</p><h3 style="margin: 1em 0 0 0;">Edge types</h3><p class="paragraph">The graph has 208 edge types, of which the 10 most common are RESEMBLES_CrC (4.60M edges, 28.97%), HAS_GhGV (2.69M edges, 16.93%), MIGHT_SUBCELLULAR_LOCATES_ImslCC (1.77M edges, 11.18%), INTERACTS_CiC (1.38M edges, 8.70%), INTERACTS_IiP (637.48K edges, 4.01%), INTERACTS_PiI (637.48K edges, 4.01%), EXPRESSES_AeG (526.18K edges, 3.31%), HAS_ChPR (404.03K edges, 2.54%), REGULATES_GrG (265.67K edges, 1.67%) and DOWNREGULATES_CHdG (214.26K edges, 1.35%). The RAM requirement for the edge types data structure is 63.54MB.</p><h4 style="margin: 1em 0 0 0;">Singleton edge types</h4><p class="paragraph">Singleton edge types are edge types that are assigned exclusively to a single edge, making the edge type relatively meaningless, as it adds no more information than the name of edge itself. The graph contains 8 edges with singleton edge types, which are IS_ACTIVE_ON_DNA_OR_RNA_LEVEL_PiaodorlCH, REGULATES_CHrCH, PRECEDING_REACTION_RLEprPW, NOT_ACTS_UPSTREAM_OF_OR_WITHIN_NEGATIVE_EFFECT_PnauoowneBP, ASSOCIATES_TO_TOXICITY_ADR_VattaCH, INCREASES_DEGENERATION_GidCH, NOT_ACTS_UPSTREAM_OF_OR_WITHIN_NEGATIVE_EFFECT_GnauoowneBP and DOWNREGULATES_GdCH. </p><h3 style="margin: 1em 0 0 0;">Topological Oddities</h3><p class="paragraph">A topological oddity is a set of nodes in the graph that <i>may be derived</i> by an error during the generation of the edge list of the graph and, depending on the task, could bias the results of topology-based models. Note that in a directed graph we only support the detection of isomorphic nodes. In the following paragraph, we will describe the detected topological oddities.</p><h4 style="margin: 1em 0 0 0;">Isomorphic node groups</h4><p class="paragraph">Isomorphic groups are nodes with exactly the same neighbours and node types (if present in the graph). Nodes in such groups are topologically indistinguishable, that is swapping their ID would not change the graph topology. We have detected 26.75K isomorphic node groups in the graph, involving a total of 212.75K nodes (8.02%) and 1.27M edges (8.00%), with the largest one involving 1.15K nodes and 6.13K edges. The detected isomorphic node groups, sorted by decreasing size, are:</p><ol class="medium-columns"><li><p class="paragraph">Group with 42 nodes (degree 146): 2145066, 2145063, 2145067, 2145068, 2145069 and other 3

## Connected holdout
Since we want to visualize an edge prediction task on this graph, we need to create a connected holdout:

In [10]:
%%time
train, test = graph.connected_holdout(train_size=0.7)
train.enable()

CPU times: user 12.4 s, sys: 195 ms, total: 12.6 s
Wall time: 6.46 s


## Compute the embedding
Next, we compute the embedding using the First-order LINE method. Do note that this implementation is a data-race aware one that uses SGD as optimizer, and nothing fancy like ADAM or NADAM: this means that the memory footprint is only limited to the embedding size.

In [11]:
%%time
from grape.embedders import FirstOrderLINEEnsmallen
embedding = FirstOrderLINEEnsmallen().fit_transform(graph)

CPU times: user 32min 1s, sys: 5.89 s, total: 32min 7s
Wall time: 1min 23s


## Visualize the embedding on the test graph
We are at the end, finally visualizing the test graph.

In [15]:
from grape import GraphVisualizer

vis = GraphVisualizer(
    graph=test,
    support=graph,
    n_components=4,
    edge_embedding_method="Hadamard",
    rotate=True,
    verbose=True,
    # Automatically, since LINE learns a cosine, the visualization tool
    # would dispatch a Cosine-distance based TSNE. This would use the sklearn
    # implementation, which is terribly slow. Therefore, we force it to use the Euclidean distance
    # and therefore the Multicore TSNE implementation (when available).
    decomposition_kwargs=dict(metric="euclidean")
)

Then we run the TSNE, this may take a while.

In [16]:
%%time
vis.fit_negative_and_positive_edges(embedding)

Performing t-SNE using 24 cores.
Using no_dims = 4, perplexity = 30.000000, and theta = 0.500000
Computing input similarities...
Building tree...
 - point 2000 of 20000
 - point 4001 of 20000
 - point 6000 of 20000
 - point 8000 of 20000
 - point 10000 of 20000
 - point 12000 of 20000
 - point 14000 of 20000
 - point 16000 of 20000
 - point 18000 of 20000
 - point 20000 of 20000
Done in 0.00 seconds (sparsity = 0.006640)!
Learning embedding...
Iteration 51: error is 101.346650 (50 iterations in 7.00 seconds)
Iteration 101: error is 85.449551 (50 iterations in 6.00 seconds)
Iteration 151: error is 80.774775 (50 iterations in 6.00 seconds)
Iteration 201: error is 79.185088 (50 iterations in 7.00 seconds)
Iteration 251: error is 78.412297 (50 iterations in 7.00 seconds)
Iteration 301: error is 2.891385 (50 iterations in 6.00 seconds)
Iteration 351: error is 2.487468 (50 iterations in 7.00 seconds)
Iteration 400: error is 2.256084 (50 iterations in 7.00 seconds)


CPU times: user 6min 45s, sys: 14min 2s, total: 20min 48s
Wall time: 53.7 s


Fitting performed in 53.00 seconds.


In [17]:
%%time
vis.plot_positive_and_negative_edges()

Rendering frames:   0%|                                                                                   | 0/…

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'


Merging frames:   0%|                                                                                     | 0/…

CPU times: user 9min 47s, sys: 12.4 s, total: 10min
Wall time: 1min 36s


In [14]:
%%time
vis.plot_edge_types()

Rendering frames:   0%|                                                                                   | 0/…

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'


Merging frames:   0%|                                                                                     | 0/…

CPU times: user 8min 8s, sys: 10.9 s, total: 8min 19s
Wall time: 1min 17s


## Cite resources
If you find useful GRAPE or PharMeBinet in your scientific work, please do recall to cite them:

###  Citing GRAPE

```bib
@misc{cappelletti2021grape,
  title={GRAPE: fast and scalable Graph Processing and Embedding},
  author={Luca Cappelletti and Tommaso Fontana and Elena Casiraghi and Vida Ravanmehr and Tiffany J. Callahan and Marcin P. Joachimiak and Christopher J. Mungall and Peter N. Robinson and Justin Reese and Giorgio Valentini},
  year={2021},
  eprint={2110.06196},
  archivePrefix={arXiv},
  primaryClass={cs.LG}
}
```

### Citing PharMeBinet

```bib
@article{konigs2022heterogeneous,
  title={The heterogeneous pharmacological medical biochemical network PharMeBINet},
  author={K{\"o}nigs, Cassandra and Friedrichs, Marcel and Dietrich, Theresa},
  journal={Scientific Data},
  volume={9},
  number={1},
  pages={1--14},
  year={2022},
  publisher={Nature Publishing Group}
}
```